In [6]:
import requests
import os
import json
import pandas as pd

from dotenv import load_dotenv

In [7]:
 # Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv('NOAA_CLIMATE_DATA')
token = api_key

In [8]:
def get_weather_data(token, location_id, start_date, end_date, data_type):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {
        'token': token
    }
    params = {
        'datasetid': 'GSOY',  # Global Historical Climatology Network - GSOM is monthly, GSOY is yearly
        'locationid': f'FIPS:{location_id}',  # Location ID for country
        'startdate': start_date,
        'enddate': end_date,
        'datatypeid': data_type,
        'units': 'standard',  # Use 'standard' for Fahrenheit, inches, etc.
        'limit': 1000  # Adjust as needed
    }

    response = requests.get(url, headers=headers, params=params)
    print(response)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code, response.text



In [9]:
# Get user input for location ID
location_id = input("Enter location ID (FIPS code): ")

# Get user input for start date
start_date = input("Enter start date (YYYY-MM-DD): ")

# Get user input for end date
end_date = input("Enter end date (YYYY-MM-DD): ")

# Date types to get
data_types = ["DP10", "DP1X", "DT32", "DX70", "DX90", "PRCP", "RHAV", "TAVG", 'TMAX']

In [10]:
# Create a dataframe from results
weather_df = pd.DataFrame(columns=['date', 'station'] + data_types)

# Loop through each data type
for data_type in data_types:
    weather_data = get_weather_data(token, location_id, start_date, end_date, data_type)
    
    # Check if the response is successful or not
    if isinstance(weather_data, tuple):
        # Error case
        print("Error occurred: Status Code:", weather_data[0], "Message:", weather_data[1])
    else:
        # Success case
        # Check if the response contains any data
        if weather_data.get("results"):
            # Parse the JSON strings into dictionaries
            results = weather_data["results"]

            # Create a dataframe from results
            weather_df = pd.DataFrame(columns=['date', 'station'] + data_types)

            # Iterate over each result
            for result in results:
                date = result["date"][:10]  # Extract date portion
                datatype = result["datatype"]
                value = result["value"]
                station = result["station"]
                
                # Check if the date already exists in the DataFrame
                if date not in weather_df['date'].tolist():
                    weather_df = weather_df.append({'date': date, 'station': station}, ignore_index=True)

                # Add the value to the corresponding cell
                weather_df.loc[weather_df['date'] == date, datatype] = value

            # Append the weather_df to the final_df
            final_df = final_df.append(weather_df, ignore_index=True)

# Constructing filename
filename = f"{location_id}_{start_date}_{end_date}_weather_data.csv"

# Export the final dataframe to a CSV file with the constructed filename
final_df.to_csv(filename, index=False)

print(f"Data saved to {filename}")

<Response [200]>


AttributeError: 'DataFrame' object has no attribute 'append'